<a href="https://colab.research.google.com/github/lovecheepao/ichiyasaGitSample/blob/main/NPD%E3%83%87%E3%83%BC%E3%82%BF%E6%95%B4%E5%BD%A2%EF%BC%88%E3%82%B9%E3%82%AD%E3%83%B3%E3%82%B1%E3%82%A2%E5%AF%BE%E5%BF%9Cver%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
# =========================================
# Google Colab 環境セットアップ
# =========================================
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/')

!pip install --upgrade gspread gspread_dataframe japanize_matplotlib openpyxl --quiet

from google.colab import auth
from google.colab import files
auth.authenticate_user()

import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe, get_as_dataframe
creds, _ = default()
gc = gspread.authorize(creds)
import pandas as pd, re
from datetime import datetime
import re
import calendar
from IPython.display import display
import numpy as np

print("✅ Colab環境セットアップ完了")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Colab環境セットアップ完了


In [ ]:
# =========================================
# Googleスプレッドシート設定
# =========================================
GSHEET_NAME = "NPD（スキンケアテスト用）"

# === 最終出力の標準列 ===
DESIRED_COLUMNS = [
    "Brand","Product",
    "Rank","Dollars","Dollars Percent Change","Dollar Share",
    "Units","Units Percent Change","Unit Share",
    "Average Price","Average Price Percent Change",
    "Segment","Subsegment","sheet_name","Period","SourceFile"
]

def _ensure_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in DESIRED_COLUMNS:
        if c not in df.columns:
            df[c] = pd.NA
    return df[DESIRED_COLUMNS]

In [ ]:
# --- シート名に基づくマッピング辞書 ---
# Makeupのシート名パターンマッピング（実際のシート名ベース）
MAKEUP_SHEET_PATTERNS = {
    #ALL
    "total makeup brands_products": ("All", "All"),

    # FACE系
    "makeup total face": ("FACE", "All"),
    "makeup total foundation": ("FACE", "FOUNDATION"),
    "makeup total blush": ("FACE", "BLUSH"),
    "makeup total bronzer": ("FACE", "BRONZER"),
    "makeup total highlighter": ("FACE", "HIGHLIGHTER"),
    "makeup total setting product": ("FACE", "SETTING SPRAY/POWDER"),
    "makeup total concealer": ("FACE", "CONCEALER"),
    "makeup total other face": ("FACE", "ALL OTHER FACE"),
    "makeup total powder": ("FACE", "POWDER"),
    "makeup total face applicator": ("FACE", "FACE APPLICATOR"),
    "makeup total face primer": ("FACE", "FACE PRIMER"),
    "makeup total tinted moisturizer": ("FACE", "TINTED MOISTURIZER"),
    "makeup total face tools & acc": ("FACE", "FACE TOOLS AND ACCESSORIES"),

    # EYE系
    "makeup total eye": ("EYE", "All"),
    "makeup total mascara": ("EYE", "MASCARA"),
    "makeup total eye shadow": ("EYE", "EYE SHADOW"),
    "makeup total eye liner": ("EYE", "EYE LINER"),
    "makeup total eye brow": ("EYE", "EYE BROW"),
    "makeup total other eye": ("EYE", "ALL OTHER EYE"),
    "makeup total eye applicator": ("EYE", "EYE APPLICATOR"),
    "makeup total eye primer": ("EYE", "EYE PRIMER"),
    "makeup total false eyelashes": ("EYE", "FALSE EYELASHES"),
    "makeup total eye tools & acc": ("EYE", "EYE TOOLS AND ACCESSORIES"),

    # LIP系
    "makeup total lip": ("LIP", "All"),
    "makeup total lip color": ("LIP", "LIP COLOR"),
    "makeup total lip gloss": ("LIP", "LIP GLOSS"),
    "makeup total lip liner": ("LIP", "LIP LINER"),
    "makeup total other lip": ("LIP", "ALL OTHER LIP"),
    "makeup total lip applicator": ("LIP", "LIP APPLICATOR"),
    "makeup total lip primer": ("LIP", "LIP PRIMER"),
    "makeup total lip tools & acc": ("LIP", "LIP TOOLS AND ACCESSORIES"),

    # SETS系
    "makeup total sets": ("SETS", "All"),
    "makeup total sets tools & acc": ("SETS", "SET TOOLS AND ACCESSORIES"),

    # ALL OTHER系
    "makeup total other color": ("ALL OTHER", "All"),
    "makeup total all other": ("ALL OTHER", "ALL OTHER"),
    "makeup total other applicator": ("ALL OTHER", "ALL OTHER APPLICATOR"),
    "makeup total all other primer": ("ALL OTHER", "ALL OTHER PRIMER"),
    "makeup all other tools & acc": ("ALL OTHER", "ALL OTHER TOOLS AND ACCESSORIES"),

    # NAIL系
    "makeup total nail": ("NAIL", "All"),
    "makeup total color enamel": ("NAIL", "COLOR ENAMEL"),
    "makeup total nail care": ("NAIL", "NAIL CARE"),
    "makeup total base_top coats": ("NAIL", "BASE COATS/TOP COATS"),
    "makeup total nail tools & acc": ("NAIL", "NAIL TOOLS AND ACCESSORIES"),
}

# Skincareのシート名パターンマッピング
SKINCARE_SHEET_PATTERNS = {
    #ALL
    "total skincare brands items": ("All", "All"),

    # FACE系
    "skincare face": ("FACE", "All"),
    "skincare all other face": ("FACE", "ALL OTHER FACE"),
    "skincare eye treatment": ("FACE", "EYE TREATMENT"),
    "skincare facial devices-tools": ("FACE", "FACIAL DEVICES/TOOLS"),
    "skincare facial cleanser": ("FACE", "FACIAL CLEANSER"),
    "skincare facial exfoliator": ("FACE", "FACIAL EXFOLIATOR"),
    "skincare lip treatment": ("FACE", "LIP TREATMENT"),
    "skincare makeup remover": ("FACE", "MAKEUP REMOVER"),
    "us skincare mask": ("FACE", "MASK"),
    "skincare shave face": ("FACE", "SHAVE TREATMENT FACE"),
    "us skincare toners-clarifyer": ("FACE", "TONER/CLARIFYER"),
    "skincare face lotion": ("FACE", "FACE LOTION"),
    "skincare face serum": ("FACE", "FACE SERUM"),
    "skincare face oil": ("FACE", "FACE OIL"),
    "skincare face cream": ("FACE", "FACE CREAM"),
    "skincare face gel": ("FACE", "FACE GEL"),
    "skincare facial spray mist": ("FACE", "FACIAL SPRAY/MIST"),
    "skincare face supplement": ("FACE", "FACE SUPPLEMENT"),

    # BODY系
    "skincare body": ("BODY", "All"),
    "skincare body cleansers": ("BODY", "BODY CLEANSER"),
    "skincare body devices-tools": ("BODY", "BODY DEVICES/TOOLS"),
    "skincare body exfoliator": ("BODY", "BODY EXFOLIATOR"),
    "skincare hand soaps": ("BODY", "HAND SOAP"),
    "skincare body cream lotion": ("BODY", "BODY CREAM/LOTION"),
    "skincare body oil": ("BODY", "BODY OIL"),
    "skincare body serum": ("BODY", "BODY SERUM"),
    "skincare body supplement": ("BODY", "BODY SUPPLEMENT"),
    "skincare body spray": ("BODY", "BODY SPRAY"),
    "skincare all other body": ("BODY", "ALL OTHER BODY"),
    "skincare deodorant": ("BODY", "DEODORANT"),
    "skincare shave body": ("BODY", "SHAVE BODY"),

    # SUN系
    "skincare sun": ("SUN", "All"),
    "skincare face in sun": ("SUN", "FACE IN SUN"),
    "skincare body in sun": ("SUN", "BODY IN SUN"),
    "us skincare self-tanner": ("SUN", "SELF-TANNER"),
    "skincare after sun": ("SUN", "AFTER SUN"),
    "skincare all other sun": ("SUN", "ALL OTHER SUN"),

    # SETS系
    "skincare sets & kits": ("SETS & KITS", "All"),
}

# Hairのシート名パターンマッピング（Skincareファイル内）
HAIR_SHEET_PATTERNS = {
    # HAIR系
    "hair shampoo_conditioner": ("SHAMPOO/CONDITIONER", "All"),
    "hair daily rinse shampoo": ("SHAMPOO/CONDITIONER", "DAILY RINSE SHAMPOO"),
    "hair daily rinse conditioner": ("SHAMPOO/CONDITIONER", "DAILY RINSE CONDITIONER"),
    "hair dry shampoo_conditioner": ("SHAMPOO/CONDITIONER", "DRY SHAMPOO/DRY CONDITIONER"),
    "hair all other shamp_condition": ("SHAMPOO/CONDITIONER", "ALL OTHER SHAMPOO OR CONDITIONER"),
    "hair treatment": ("TREATMENT", "All"),
    "hair mask": ("TREATMENT", "MASK"),
    "hair oil": ("TREATMENT", "OIL"),
    "hair serum": ("TREATMENT", "SERUM"),
    "leave-in scalp treatment": ("TREATMENT", "LEAVE-IN/SCALP TREATMENT"),
    "hair all other treatment": ("TREATMENT", "ALL OTHER TREATMENT"),
    "hair color": ("COLOR", "All"),
    "hair color subseg": ("COLOR", "COLOR"),
    "hair root touch up_glaze": ("COLOR", "ROOT TOUCH UP"),
    "toner_glaze": ("COLOR", "TONER/GLAZE"),
    "hair sets": ("SETS", "All"),
    "hair styling": ("STYLING", "All"),
    "hair mousse foam": ("STYLING", "MOUSSE/FOAM"),
    "hair gel_wax_pomade": ("STYLING", "GEL/WAX/POMADE"),
    "hair styling primer_heat protec": ("STYLING", "PRIMER/HEAT PROTECTANT"),
    "hair cream_lotion": ("STYLING", "STYLING CREAM/LOTION"),
    "hair spray": ("STYLING", "SPRAY"),
    "hair all other styling": ("STYLING", "ALL OTHER STYLING"),
}

#Total Makeupシートのマッピング辞書
sub_to_segment_makeup = {
    # FACE
    "FOUNDATION": "FACE",
    "CONCEALER": "FACE",
    "SETTING SPRAY/POWDER": "FACE",
    "BLUSH": "FACE",
    "ALL OTHER FACE": "FACE",
    "TINTED MOISTURIZER": "FACE",
    "FACE PRIMER": "FACE",
    "FACE APPLICATOR": "FACE",
    "HIGHLIGHTER": "FACE",
    "BRONZER": "FACE",
    "POWDER": "FACE",
    "FACE TOOLS AND ACCESSORIES": "FACE",

    # EYE
    "MASCARA": "EYE",
    "EYE BROW": "EYE",
    "EYE SHADOW": "EYE",
    "EYE LINER": "EYE",
    "EYE TOOLS AND ACCESSORIES": "EYE",
    "EYE PRIMER": "EYE",
    "FALSE EYELASHES": "EYE",
    "EYE APPLICATOR": "EYE",
    "ALL OTHER EYE": "EYE",

    # LIP
    "ALL OTHER LIP": "LIP",
    "LIP COLOR": "LIP",
    "LIP GLOSS": "LIP",
    "LIP LINER": "LIP",
    "LIP TOOLS AND ACCESSORIES": "LIP",
    "LIP PRIMER": "LIP",
    "LIP APPLICATOR": "LIP",

    # SETS
    "SET": "SETS",
    "SET TOOLS AND ACCESSORIES": "SETS",

    # ALL OTHER
    "OTHER COLOR": "ALL OTHER",
    "ALL OTHER": "ALL OTHER",
    "ALL OTHER APPLICATOR": "ALL OTHER",
    "ALL OTHER TOOLS AND ACCESSORIES": "ALL OTHER",
    "ALL OTHER PRIMER": "ALL OTHER",

    # NAIL
    "COLOR ENAMEL": "NAIL",
    "NAIL TOOLS AND ACCESSORIES": "NAIL",
    "NAIL CARE": "NAIL",
    "BASE COATS/TOP COATS": "NAIL",
}

# Skincareのマッピング辞書
sub_to_segment_skincare = {
    # FACE
    "FACE CREAM": "FACE",
    "FACE SERUM": "FACE",
    "FACIAL CLEANSER": "FACE",
    "EYE TREATMENT": "FACE",
    "ALL OTHER FACE": "FACE",
    "TONER/CLARIFYER": "FACE",
    "FACIAL EXFOLIATOR": "FACE",
    "FACE LOTION": "FACE",
    "LIP TREATMENT": "FACE",
    "MASK": "FACE",
    "FACE OIL": "FACE",
    "FACIAL SPRAY/MIST": "FACE",
    "FACE GEL": "FACE",
    "MAKEUP REMOVER": "FACE",
    "FACIAL DEVICES/TOOLS": "FACE",
    "SHAVE TREATMENT FACE": "FACE",
    "FACE SUPPLEMENT": "FACE",

    # BODY
    "BODY CREAM/LOTION": "BODY",
    "BODY CLEANSER": "BODY",
    "BODY SUPPLEMENT": "BODY",
    "HAND SOAP": "BODY",
    "BODY OIL": "BODY",
    "ALL OTHER BODY": "BODY",
    "DEODORANT": "BODY",
    "BODY EXFOLIATOR": "BODY",
    "BODY SPRAY": "BODY",
    "BODY DEVICES/TOOLS": "BODY",
    "BODY SERUM": "BODY",
    "SHAVE BODY": "BODY",

    # SUN
    "FACE IN SUN": "SUN",
    "BODY IN SUN": "SUN",
    "SELF-TANNER": "SUN",
    "AFTER SUN": "SUN",
    "ALL OTHER SUN": "SUN",

    # SETS & KITS
    "SETS/KITS": "SETS & KITS",
}

# Hairシートのマッピング辞書
sub_to_segment_hair = {
    # SHAMPOO/CONDITIONER
    "DAILY RINSE SHAMPOO": "SHAMPOO/CONDITIONER",
    "DAILY RINSE CONDITIONER": "SHAMPOO/CONDITIONER",
    "DRY SHAMPOO/DRY CONDITIONER": "SHAMPOO/CONDITIONER",
    "ALL OTHER SHAMPOO OR CONDITIONER": "SHAMPOO/CONDITIONER",

    # TREATMENT（元のNPDデータに間違いあり、、）
    "MASK": "TREATMENT",
    "OIL": "TREATMENT",
    "SERUM": "TREATMENT",
    "LEAVE-IN/SCALP TREATMENT": "TREATMENT",
    "ALL OTHER TREATMENT": "TREATMENT",

    # COLOR
    "COLOR": "COLOR",
    "ROOT TOUCH UP": "COLOR",
    "TONER/GLAZE": "COLOR",

    # SETS
    "SETS": "SETS",

    # STYLING
    "ALL OTHER STYLING": "STYLING",
    "SPRAY": "STYLING",
    "STYLING CREAM/LOTION": "STYLING",
    "PRIMER/HEAT PROTECTANT": "STYLING",
    "GEL/WAX/POMADE": "STYLING",
    "MOUSSE/FOAM": "STYLING",
}

#表形式が特殊なシート
SPECIAL_CATEGORY_SHEETS = {
    "total makeup", "total skincare", "hair"
}

#ドメインを固定したいシート
DOMAIN_SHEET_OVERRIDES = {
    # 正規化（小文字・連続空白1化）後のシート名で指定
    "toner_glaze": "hair",
    "skincare body in sun": "skincare"
}

In [ ]:
# ============================================================
# Colab: 全シート→4表抽出 & Segment/Subsegment 付与
# Makeup/Skincare/Hair 対応 + 複数ファイル + プレフィックス付き出力
# 出力シート名: {domain}_{table}_{period}
#   例) makeup_brand_month / skincare_product_ytd / hair_category_month
# ============================================================

# ---------------- 0) Imports & フォールバック辞書 ----------------
import re
import os
import pandas as pd
from typing import Dict, List
from google.colab import files

# すでに別セルで定義済みならその値を使い、未定義なら空辞書でフォールバック
try:
    MAKEUP_SHEET_PATTERNS
except NameError:
    MAKEUP_SHEET_PATTERNS = {}

try:
    SKINCARE_SHEET_PATTERNS
except NameError:
    SKINCARE_SHEET_PATTERNS = {}

try:
    HAIR_SHEET_PATTERNS
except NameError:
    HAIR_SHEET_PATTERNS = {}

try:
    sub_to_segment_makeup
except NameError:
    sub_to_segment_makeup = {}

try:
    sub_to_segment_skincare
except NameError:
    sub_to_segment_skincare = {}

try:
    sub_to_segment_hair
except NameError:
    sub_to_segment_hair = {}

# ---------------- 1) 設定 ----------------
GSHEET_OUTPUT = False  # TrueにするとGSheetsへも出力（gc等の認証が別セルで済んでいる前提）
GSHEET_NAME   = "NPD（スキンケアテスト用）"  # 元セルの値を反映

# ドメイン別 YTD オフセット（必要なら調整）
YTD_OFFSET_MAP = {"makeup": 2, "skincare": 2, "hair": 2}

# ---------------- 2) ユーティリティ ----------------
def extract_date_from_filename(filename: str):
    """ファイル名 'March 2025' のような表記から '2025-03' を返す。未抽出なら None。"""
    m = re.search(r'(January|February|March|April|May|June|July|August|September|October|November|December)\s+(\d{4})', filename)
    if not m:
        return None
    month_str, year_str = m.groups()
    month_map = {
        "January": 1, "February": 2, "March": 3, "April": 4, "May": 5,
        "June": 6, "July": 7, "August": 8, "September": 9,
        "October": 10, "November": 11, "December": 12
    }
    return f"{year_str}-{month_map[month_str]:02d}"

def normalize_sheet_name(name: str) -> str:
    return re.sub(r'\s+', ' ', (name or "").lower().strip())

def detect_domain(sheet_or_file_name: str) -> str:
    n = (sheet_or_file_name or "").lower()
    if "skincare" in n or n.startswith("skincare"):
        return "skincare"
    if n.startswith("hair") or "scalp" in n:
        return "hair"
    return "makeup"

def normalize_part_key(raw_key: str, domain: str) -> str:
    """
    "brand_month" / "product_ytd" / "category_month"といったキーを
    {domain}_{table}_{period} に正規化する。
    """
    k = (raw_key or "").lower().strip()
    for d in ("makeup_", "skincare_", "hair_"):
        if k.startswith(d):
            k = k[len(d):]

    if   k.startswith("brand_"):
        table, suffix = "brand",   k[len("brand_"):]
    elif k.startswith("product_"):
        table, suffix = "product", k[len("product_"):]
    elif k.startswith("category_"):
        table, suffix = "category", k[len("category_"):]
    else:
        raise ValueError(f"Unexpected part key: {raw_key}")

    if suffix not in ("month", "ytd"):
        raise ValueError(f"Unexpected period in part key: {raw_key}")

    return f"{domain}_{table}_{suffix}"

# ---------------- 3) Segment/Subsegment 抽出 ----------------
def is_valid_partial_match(sheet_name, pattern):
    if pattern in sheet_name:
        return True
    p_words = set(pattern.split())
    s_words = set(sheet_name.split())
    important = p_words - {'total', 'makeup', 'skincare', 'hair'}
    return bool(important) and important.issubset(s_words)

def fallback_mapping(norm, orig):
    # 未マッチ時の戻り値（Unknown で継続）
    return ("Unknown", "Unknown")

def extract_segment_subsegment(sheet_name, is_skincare=False):
    n = normalize_sheet_name(sheet_name)
    if n.startswith('hair') or 'scalp' in n:
        patterns = HAIR_SHEET_PATTERNS
    elif is_skincare:
        patterns = SKINCARE_SHEET_PATTERNS
    else:
        patterns = MAKEUP_SHEET_PATTERNS

    norm_patterns = {}
    for p, m in patterns.items():
        p2 = normalize_sheet_name(p)
        norm_patterns[p2] = m

    for pat, (seg, sub) in sorted(norm_patterns.items(), key=lambda x: len(x[0]), reverse=True):
        if pat == n or is_valid_partial_match(n, pat):
            return seg, sub
    return fallback_mapping(n, sheet_name)

def fill_segment_subsegment(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    def _infer_from_sheetname(sname: str):
        if not sname:
            return (None, None)
        key = normalize_sheet_name(sname)

        def _lookup(key, mapping):
            # 完全一致→前方一致
            if key in mapping:
                return mapping[key]
            for k, v in mapping.items():
                if key.startswith(normalize_sheet_name(k)):
                    return v
            return (None, None)

        for mapping in (MAKEUP_SHEET_PATTERNS, SKINCARE_SHEET_PATTERNS, HAIR_SHEET_PATTERNS):
            seg, sub = _lookup(key, mapping)
            if seg or sub:
                return (seg, sub)
        return (None, None)

    if "sheet_name" in out.columns:
        seg_sub = out["sheet_name"].astype(str).map(lambda s: _infer_from_sheetname(s))
        seg_series  = seg_sub.map(lambda x: x[0])
        sub_series  = seg_sub.map(lambda x: x[1])

        if "Segment" not in out.columns:
            out["Segment"] = seg_series
        else:
            out["Segment"] = out["Segment"].fillna(seg_series)

        if "Subsegment" not in out.columns:
            out["Subsegment"] = sub_series
        else:
            out["Subsegment"] = out["Subsegment"].fillna(sub_series)

    # Subsegment だけが埋まって Segment が空なら、ドメイン別の辞書で補完
    if "Segment" in out.columns and "Subsegment" in out.columns:
        # 行ごとに sheet_name からドメインを推定して適切な辞書を選ぶ
        def _map_sub_to_seg(row):
            seg = row.get("Segment")
            sub = row.get("Subsegment")
            if pd.isna(seg) and pd.notna(sub):
                sname = str(row.get("sheet_name", ""))
                domain = detect_domain(sname)  # "makeup" | "skincare" | "hair"
                sub_key = str(sub).upper()

                if domain == "skincare" and sub_to_segment_skincare:
                    return sub_to_segment_skincare.get(sub_key, seg)
                elif domain == "hair" and sub_to_segment_hair:
                    return sub_to_segment_hair.get(sub_key, seg)
                elif sub_to_segment_makeup:
                    return sub_to_segment_makeup.get(sub_key, seg)
            return seg

        out["Segment"] = out.apply(_map_sub_to_seg, axis=1)

    # --- TOTAL 接頭辞と Grand Total の補正 ---
    if "Segment" in out.columns:
        seg = out["Segment"].astype(str).str.strip()

        # 1) TOTAL ○○ → Segment=○○, Subsegment=All
        mask_total = seg.str.upper().str.startswith("TOTAL ")
        out.loc[mask_total, "Segment"] = seg[mask_total].str.replace(r"^TOTAL\s+", "", regex=True)
        out.loc[mask_total, "Subsegment"] = "All"

        # 2) Subsegment が空欄なら All
        mask_blank = out["Subsegment"].isna() | (out["Subsegment"].astype(str).str.strip() == "")
        out.loc[mask_blank, "Subsegment"] = "All"

    return out

# ---------------- 4) ヘッダー検出 / 整形 ----------------
def find_header_row(df):
    pat = re.compile(r'(brand|product|rank|dollar|ytd)', re.I)
    for i in range(len(df)):
        if pat.search(" ".join(df.iloc[i].astype(str).str.lower())):
            return i
    return 0

def find_product_row(df, start, kw=r"(products?)", max_scan=300):
    pat = re.compile(kw, re.I)
    for r in range(start, min(len(df), max_scan)):
        if pat.search(" ".join(df.iloc[r].astype(str).str.lower())):
            return r
    return None

def find_ytd_col(df, hdr, scan_rows=3):
    for r in range(hdr, min(hdr + scan_rows, len(df))):
        for c, v in enumerate(df.iloc[r]):
            if isinstance(v, str) and 'ytd' in v.lower():
                return c
    return None

def tidy(df, scan_rows=6):
    if df.empty:
        return df
    z = df.dropna(axis=1, how='all').copy()

    z_scan = z.astype(str).apply(lambda s: s.str.strip() if s.dtype == "object" else s)

    # ← brand/product だけでなく segment/subsegment も見る
    pat = re.compile(r'\b(brand|brands?|product|products?|item|items?|'
                     r'segment|subsegment|rank|dollar|dollars|units?|ytd)\b', re.I)

    header_row = 0
    for i in range(min(scan_rows, len(z_scan))):
        row_txt = " ".join(z_scan.iloc[i].tolist()).lower()
        if pat.search(row_txt):
            header_row = i
            break

    cols_raw = list(z.iloc[header_row])
    # 空名→""、重複は _1, _2
    cols_norm, seen = [], {}
    for c in cols_raw:
        name = "" if (pd.isna(c) or str(c).strip().lower() in {"", "nan", "none", "-"}) else str(c).strip()
        if name not in seen:
            seen[name] = 0; cols_norm.append(name)
        else:
            seen[name] += 1; cols_norm.append(f"{name}_{seen[name]}")
    z.columns = cols_norm

    z = z.iloc[header_row + 1:].reset_index(drop=True)

    # 先頭が全空様なら一行だけ落とす（安全側）
    def _empty_like_row(s): return s.astype(str).str.strip().str.lower().isin(["", "nan", "none", "-"]).all()
    if len(z) and _empty_like_row(z.iloc[0]): z = z.iloc[1:].reset_index(drop=True)

    return z

def drop_rows_only_subsegment(df: pd.DataFrame) -> pd.DataFrame:
    """
    Subsegment が入っていて、かつ Units と Dollar Share が両方とも空、
    さらに「ラベル列（Brand/Brands/Product/Products/Items のいずれか）」も空の行だけ削除。
    該当ラベル列が見つからない場合は安全側で何もしない。
    """
    if df is None or df.empty:
        return df

    z = df.copy()

    # Share / Units の列（最小ゆらぎ）
    share_col = "Dollar Share" if "Dollar Share" in z.columns else ("Dollars Share" if "Dollars Share" in z.columns else None)
    units_col = "Units" if "Units" in z.columns else None
    if not units_col or not share_col:
        return z  # どちらか無ければ何もしない（安全側）

    def _is_empty_val(x) -> bool:
        if pd.isna(x):
            return True
        s = str(x).strip().lower()
        return s in {"", "nan", "none"}

    # Subsegment が非空
    has_sub   = ~z.get("Subsegment", pd.Series("", index=z.index)).astype(str).str.strip().eq("")
    # 値が空か
    units_emp = z[units_col].apply(_is_empty_val)
    share_emp = z[share_col].apply(_is_empty_val)

    # --- ラベル列（Brand/Brands/Product/Products/Items のいずれかを優先順で採用） ---
    label_candidates = []
    for col in z.columns:
        name = str(col).strip().lower()
        if re.search(r'\bbrand(s)?\b', name) or re.search(r'\bproduct(s)?\b', name) or re.search(r'\bitems?\b', name):
            label_candidates.append(col)

    if not label_candidates:
        # ラベル列が見当たらない場合は何もしない（勝手に落とさない）
        return z

    # 1本に決める（最初の候補を使う）
    label_col = label_candidates[0]
    label_emp = z[label_col].apply(_is_empty_val)

    # すべての条件を満たす行のみ drop
    drop_mask = has_sub & units_emp & share_emp & label_emp

    return z.loc[~drop_mask].reset_index(drop=True)

# ---------------- 5) テーブル分割（左右：month/YTD × 上下：brand/product） ----------------
def split_into_4(raw_df: pd.DataFrame, ytd_offset: int = 2, min_blank_cols: int = 2) -> Dict[str, pd.DataFrame]:
    """
    基本の表（左=単月 右=YTD、上段=Brand 下段=Product）を4つに分割して返す。
    ytd_offset: YTD開始列の補正（1 or 2 など）
    """
    df = raw_df.copy()
    df = df.dropna(axis=1, how='all')
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    hdr = find_header_row(df)
    prod_row = find_product_row(df, hdr + 1) or len(df)
    ytd_col  = find_ytd_col(df, hdr)

    if ytd_col is not None:
        col_bd = max(ytd_col - ytd_offset, 0)
    else:
        # 明示的にYTDヘッダーが見つからない場合は「空列が連続した位置」を境界とする
        blank, in_table, col_bd = 0, False, df.shape[1]
        for c in range(df.shape[1]):
            col = df.iloc[:, c]
            if not in_table and not col.isna().all():
                in_table = True
            if in_table and (col.isna().all() or col.eq('').all()):
                blank += 1
                if blank >= min_blank_cols:
                    col_bd = c - blank + 1
                    break
            else:
                blank = 0

    # まず左右に分割（行は暫定的に左側のhdr/prod_rowに合わせる）
    brand_month   = df.iloc[hdr:prod_row, :col_bd].reset_index(drop=True)
    brand_ytd_raw = df.iloc[hdr:prod_row, col_bd:].reset_index(drop=True)
    product_month = df.iloc[prod_row:,  :col_bd].reset_index(drop=True)
    product_ytd   = df.iloc[prod_row:,   col_bd:].reset_index(drop=True)

    # --- 右上（brand_ytd）だけヘッダー段差を補正 ------------------------
    def _trim_to_header(d: pd.DataFrame) -> pd.DataFrame:
        # ヘッダーに現れやすいトークン（Circana/NPD系）
        tokens = {
            "segment", "subsegment",
            "dollars", "dollars percent change", "dollar share",
            "units", "units percent change", "unit share",
            "average price", "average price percent change"
        }
        max_scan = min(6, len(d))  # 先頭〜数行だけスキャン
        best_i, best_hits = 0, -1
        for i in range(max_scan):
            row = d.iloc[i].astype(str).str.lower()
            hits = sum(any(tok in cell for cell in row) for tok in tokens)
            if hits > best_hits:
                best_hits = hits
                best_i = i
        # “ヘッダーっぽい行”が先頭より下にあれば、そこから切り直す
        if best_i > 0 and best_hits >= 2:
            return d.iloc[best_i:].reset_index(drop=True)
        # すべてNaNの先頭行があれば落としておく（保険）
        j = 0
        while j < len(d) and d.iloc[j].isna().all():
            j += 1
        return d.iloc[j:].reset_index(drop=True)

    brand_ytd = _trim_to_header(brand_ytd_raw)
    # -------------------------------------------------------------------

    return {
        "brand_month":   brand_month,
        "brand_ytd":     brand_ytd,
        "product_month": product_month,
        "product_ytd":   product_ytd,
    }

# ---------------- 6) 特殊カテゴリ表（上下分割固定レイアウト） ----------------
#ドメイン別に固定スライス
def split_category_table_fixed(df, domain: str):
    """
    カテゴリ表を固定スライスで分割（ドメイン別に開始位置を固定）
    スライスは tidy() に渡すため、ヘッダー行を含む範囲にすること。
    行番号は 0-indexed で iloc に渡す（右端は非包含）。
    """
    df = df.dropna(axis=1, how='all')

    if domain == "skincare":
        # 月次: ヘッダー=4行目, データ=5～44行
        # YTD:  ヘッダー=51行目, データ=52～91行
        month_slice = df.iloc[3:44]
        ytd_slice   = df.iloc[50:91]
    elif domain == "hair":
        # 月次: ヘッダー=4行目, データ=5～28行
        # YTD:  ヘッダー=35行目, データ=36～59行
        month_slice = df.iloc[3:28]
        ytd_slice   = df.iloc[34:59]
    else:
        # 月次: ヘッダー=4行目, データ=5～49行
        # YTD:  ヘッダー=52行目, データ=36～97行
        month_slice = df.iloc[3:49]
        ytd_slice   = df.iloc[51:97]

    # どちらも必ず DataFrame（空でも）を返す
    month_df = month_slice.copy()
    ytd_df   = ytd_slice.copy()
    return {"category_month": month_df, "category_ytd": ytd_df}

# ---------------- 7) メイン処理：複数ファイル → 全シート走査 ----------------
uploaded = files.upload()  # 複数選択OK
all_agg: Dict[str, List[pd.DataFrame]] = {}

skip_patterns = [
    'header', 'cover', 'methodology', 'total beauty',
    'makeup franchises', 'makeup launches', 'inventory',
    'hair_upgrade', 'skincare mapping glossary',
    "total women's skincare", "total men's skincare",
    'skincare franchise', 'skincare launches','skincare mapping glossary','skincare acne treatment','skincare age specialist',
    'skincare brightening spec','skincare facial moisturizer','skincare oil-shine control'
]

offset1_sheets = set()  # 例: {"makeup total concealer", ...} 必要なら正規化名で追加

for xlsx_path in uploaded.keys():
    period_str = extract_date_from_filename(xlsx_path)
    xls = pd.ExcelFile(xlsx_path)

    for sh in xls.sheet_names:
        norm = normalize_sheet_name(sh)
        if any(pat in norm for pat in skip_patterns):
            print(f"⏭️  skip sheet: {sh}")
            continue

        raw = pd.read_excel(xls, sheet_name=sh, header=None)
        print(f"📄 処理中: {os.path.basename(xlsx_path)} :: {sh} - shape: {raw.shape}")

        # ドメイン推定（シート名優先、ダメならファイル名でフォールバック）
        domain = detect_domain(sh)
        norm = normalize_sheet_name(sh)

        # ★ 上書き辞書を最優先で適用
        if norm in DOMAIN_SHEET_OVERRIDES:
            domain = DOMAIN_SHEET_OVERRIDES[norm]
        else:
            # 既存のフォールバックは必要な場合のみ
            if domain == "makeup":
                domain = detect_domain(os.path.basename(xlsx_path)) or domain

        # テーブル抽出
        if norm in SPECIAL_CATEGORY_SHEETS:
            print(f"📊 特別カテゴリ形式で処理: {sh}")
            tables = split_category_table_fixed(raw, domain)   # ← domain を渡す
        else:
            offset = 1 if norm in offset1_sheets else YTD_OFFSET_MAP.get(domain, 2)
            tables = split_into_4(raw, ytd_offset=offset, min_blank_cols=2)

        # セグメント推定（sheet名から）
        is_sk = (domain == "skincare")
        seg, sub = extract_segment_subsegment(sh, is_skincare=is_sk)

        # 集約
        for raw_key, d in tables.items():
            if d is None:
                continue

            d = tidy(d)
            if d is None or not isinstance(d, pd.DataFrame) or d.empty:
                continue
            if d.empty:
                print(f"[DBG-agg] {sh} :: {raw_key} → empty after tidy")
                continue

            key = normalize_part_key(raw_key, domain)

            # ★ ここを先に：出所情報を埋めてから fill_segment_subsegment を呼ぶ
            d["sheet_name"] = sh
            d["SourceFile"] = os.path.basename(xlsx_path)
            if "Period" not in d.columns or d["Period"].isna().all():
                d["Period"] = period_str

            # ★ Segment/Subsegment は辞書で一括補完（category/brand/product 全て）
            #   ※ domain ヒントを渡したい場合は fill_segment_subsegment(d, domain_hint=domain)
            d = fill_segment_subsegment(d)

            # ★ brand/product だけ、値の無い“Subsegmentだけ行”を落とす
            if raw_key.startswith("brand_") or raw_key.startswith("product_"):
                d = drop_rows_only_subsegment(d)
                if d is None or d.empty:
                    print(f"[DBG-drop] {sh} :: {raw_key} → empty after drop", flush=True)
                    continue

            # ★ 旧ロジック（seg/sub を直接代入）は不要なので削除
            # if "_category_" not in key: ... （ここは消す）

            all_agg.setdefault(key, []).append(d)


# ---------------- 8) 結合・カテゴリ補完・保存 ----------------
dfs = {k: (pd.concat(v, ignore_index=True) if v else pd.DataFrame()) for k, v in all_agg.items()}

# category系は辞書で補完
for k in list(dfs.keys()):
    if "_category_" in k and not dfs[k].empty:
        dfs[k] = fill_segment_subsegment(dfs[k])

# プレビュー
for k in sorted(dfs.keys()):
    print(f"\n--- {k} (head) ---")
    display(dfs[k].head(3))

Saving 3 - Skincare Data Workbook March 2025 YTD.xlsx to 3 - Skincare Data Workbook March 2025 YTD (27).xlsx
Saving 4 - Makeup Data Workbook April 2025 YTD.xlsx to 4 - Makeup Data Workbook April 2025 YTD (30).xlsx
Saving 5 - Makeup Data Workbook May 2025 YTD.xlsx to 5 - Makeup Data Workbook May 2025 YTD (17).xlsx
Saving 6 - Makeup Data Workbook June 2025 YTD.xlsx to 6 - Makeup Data Workbook June 2025 YTD (10).xlsx
⏭️  skip sheet: Inventory
⏭️  skip sheet: hair_upgrade
⏭️  skip sheet: header
⏭️  skip sheet: Cover
⏭️  skip sheet: Methodology
⏭️  skip sheet: Total Beauty
📄 処理中: 3 - Skincare Data Workbook March 2025 YTD (27).xlsx :: Total Skincare - shape: (91, 10)
📊 特別カテゴリ形式で処理: Total Skincare
⏭️  skip sheet: Total Women's Skincare
⏭️  skip sheet: Total Men's Skincare
⏭️  skip sheet: Skincare Franchise
⏭️  skip sheet: Skincare Launches
📄 処理中: 3 - Skincare Data Workbook March 2025 YTD (27).xlsx :: Total Skincare Brands Items - shape: (85, 21)
📄 処理中: 3 - Skincare Data Workbook March 2025 YT

/tmp/ipython-input-1451440824.py:324: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/tmp/ipython-input-1451440824.py:324: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/tmp/ipython-input-1451440824.py:324: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


⏭️  skip sheet: SKINCARE ACNE TREATMENT
⏭️  skip sheet: SKINCARE AGE SPECIALIST
📄 処理中: 3 - Skincare Data Workbook March 2025 YTD (27).xlsx :: SKINCARE ALL OTHER FACE - shape: (85, 21)
⏭️  skip sheet: SKINCARE BRIGHTENING SPEC
📄 処理中: 3 - Skincare Data Workbook March 2025 YTD (27).xlsx ::  SKINCARE EYE TREATMENT - shape: (85, 21)
📄 処理中: 3 - Skincare Data Workbook March 2025 YTD (27).xlsx :: SKINCARE FACIAL DEVICES-TOOLS - shape: (85, 21)


/tmp/ipython-input-1451440824.py:324: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/tmp/ipython-input-1451440824.py:324: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/tmp/ipython-input-1451440824.py:324: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


📄 処理中: 3 - Skincare Data Workbook March 2025 YTD (27).xlsx :: SKINCARE FACIAL CLEANSER - shape: (85, 21)
📄 処理中: 3 - Skincare Data Workbook March 2025 YTD (27).xlsx :: SKINCARE FACIAL EXFOLIATOR - shape: (85, 21)
⏭️  skip sheet: SKINCARE FACIAL MOISTURIZER
📄 処理中: 3 - Skincare Data Workbook March 2025 YTD (27).xlsx :: SKINCARE LIP TREATMENT - shape: (60, 21)


...

,Segment,Subsegment,Dollars,Dollars Percent Change,Dollar Share,Units,Units Percent Change,Unit Share,Average Price,Average Price Percent Change,sheet_name,SourceFile,Period
0,Grand Total,All,2145537895.22409,-2.6146,100,84012375,1.2251,100,25.538355,-3.7933,Total Skincare,3 - Skincare Data Workbook March 2025 YTD (27)...,2025-03
1,FACE,All,1404461992.07384,-3.3704,65.4597,51518316,-2.055,61.3223,27.261411,-1.343,Total Skincare,3 - Skincare Data Workbook March 2025 YTD (27)...,2025-03
2,FACE,FACE CREAM,357852480.560207,-1.0883,16.6789,7443914,-0.9909,8.8605,48.073162,-0.0984,Total Skincare,3 - Skincare Data Workbook March 2025 YTD (27)...,2025-03



--- skincare_product_month (head) ---


,Product,Rank,Dollars,Dollars Percent Change,Dollar Share,Units,Units Percent Change,Unit Share,Average Price,Average Price Percent Change,sheet_name,SourceFile,Period,Segment,Subsegment
0,Grand Total,NaN,909763390.416366,0.2677,100,34810018,4.1046,100,26.135103,-3.6856,Total Skincare Brands Items,3 - Skincare Data Workbook March 2025 YTD (27)...,2025-03,All,All
1,THE ORDINARY GLYC ACID 7% TONING 8 SLTN,1,4566883.71203,11.3133,0.502,364999,10.6816,1.0485,12.512044,0.5708,Total Skincare Brands Items,3 - Skincare Data Workbook March 2025 YTD (27)...,2025-03,All,All
2,THE ORDINARY HYALURONIC ACID 2%+B5 HYD 1 SRM,2,3213471.426907,-30.0929,0.3532,334911,-30.4363,0.9621,9.595001,0.4937,Total Skincare Brands Items,3 - Skincare Data Workbook March 2025 YTD (27)...,2025-03,All,All



--- skincare_product_ytd (head) ---


,Product,Rank,Dollars,Dollars Percent Change,Dollar Share,Units,Units Percent Change,Unit Share,Average Price,Average Price Percent Change,sheet_name,SourceFile,Period,Segment,Subsegment
0,Grand Total,NaN,2145537895.22409,-2.6146,100,84012375,1.2251,100,25.538355,-3.7933,Total Skincare Brands Items,3 - Skincare Data Workbook March 2025 YTD (27)...,2025-03,All,All
1,THE ORDINARY GLYC ACID 7% TONING 8 SLTN,1,11324476.12739,21.3577,0.5278,901468,20.5925,1.073,12.562261,0.6345,Total Skincare Brands Items,3 - Skincare Data Workbook March 2025 YTD (27)...,2025-03,All,All
2,THE ORDINARY HYALURONIC ACID 2%+B5 HYD 1 SRM,2,8533290.177257,32.1974,0.3977,889653,31.85,1.059,9.591706,0.2635,Total Skincare Brands Items,3 - Skincare Data Workbook March 2025 YTD (27)...,2025-03,All,All


In [ ]:
# 列名重複チェック（all_agg でも dfs でも使える汎用版）
def check_duplicate_columns(obj, name="(unknown)", preview=0):
    """
    obj: dict[str, List[pd.DataFrame]]  もしくは  dict[str, pd.DataFrame]
         - 前者: all_agg（分割直後のリスト）
         - 後者: dfs（結合後の最終テーブル）
    name: ログに出す対象名（例: "all_agg", "dfs"）
    preview: 問題があった場合に先頭数行を表示する行数（0なら表示しない）
    """
    has_error = False

    def _dup_list(cols):
        # keep=False で重複している全ての列名を拾う（順序保持）
        mask = cols.duplicated(keep=False)
        if not mask.any():
            return []
        # unique順保持で抽出
        seen, out = set(), []
        for c, m in zip(cols, mask):
            if m and c not in seen:
                seen.add(c)
                out.append(c)
        return out

    for key, val in obj.items():
        # val がリストなら pre-concat（all_agg）、DataFrame なら post-concat（dfs）
        if isinstance(val, list):
            for idx, df in enumerate(val):
                if df is None or df.empty:
                    continue
                dup_cols = _dup_list(pd.Index(df.columns))
                if dup_cols:
                    has_error = True
                    sheet_name = df["sheet_name"].iloc[0] if "sheet_name" in df.columns and len(df) else "(不明)"
                    src = df["SourceFile"].iloc[0] if "SourceFile" in df.columns and len(df) else "(不明)"
                    print(f"⚠️[{name}] key='{key}' / idx={idx} / sheet='{sheet_name}' / source='{src}' / shape={df.shape}")
                    print(f"   重複列: {dup_cols}")
                    if preview > 0:
                        display(df.head(preview))
                    print("-" * 60)
        else:
            df = val
            if df is None or (isinstance(df, pd.DataFrame) and df.empty):
                continue
            dup_cols = _dup_list(pd.Index(df.columns))
            if dup_cols:
                has_error = True
                sheet_name = df["sheet_name"].iloc[0] if "sheet_name" in df.columns and len(df) else "(不明)"
                src = df["SourceFile"].iloc[0] if "SourceFile" in df.columns and len(df) else "(不明)"
                print(f"⚠️[{name}] key='{key}' / sheet='{sheet_name}' / source='{src}' / shape={df.shape}")
                print(f"   重複列: {dup_cols}")
                if preview > 0:
                    display(df.head(preview))
                print("-" * 60)

    if not has_error:
        print(f"✅[{name}] 重複列は見つかりませんでした。")

# 実行例
check_duplicate_columns(all_agg, name="all_agg", preview=0)  # 分割直後のチェック
check_duplicate_columns(dfs, name="dfs", preview=0)          # 結合後のチェック

✅[all_agg] 重複列は見つかりませんでした。
✅[dfs] 重複列は見つかりませんでした。


In [ ]:
def append_to_gsheet_with_dedup_v2(dfs_dict, spreadsheet_name):
    """
    追記（インクリメンタル）+ 重複除去 + 列順統一 + シート種別対応（category/brand/product）
    - dfs_dict: {sheet_name: DataFrame}
      例) "makeup_category_ytd", "hair_brand_month", "skincare_product_month" など
    """
    import gspread
    from gspread_dataframe import set_with_dataframe, get_as_dataframe
    import pandas as pd

    # 出力列（存在する列だけをこの順に出力する。sheet_name は必ず補完して出力）
    desired_columns = [
        "Brand", "Product",
        "Rank", "Dollars", "Dollars Percent Change", "Dollar Share",
        "Units", "Units Percent Change", "Unit Share",
        "Average Price", "Average Price Percent Change",
        "Segment", "Subsegment", "sheet_name", "Period"
    ]
    numeric_candidates = [
        "Rank", "Dollars", "Dollars Percent Change", "Dollar Share",
        "Units", "Units Percent Change", "Unit Share",
        "Average Price", "Average Price Percent Change"
    ]

    # スプレッドシートを開く or 作成
    try:
        sh = gc.open(spreadsheet_name)
        print(f"📗 Opened: {spreadsheet_name}")
    except gspread.SpreadsheetNotFound:
        sh = gc.create(spreadsheet_name)
        print(f"🆕 Created spreadsheet: {spreadsheet_name}")

    def _sheet_kind(name: str) -> str:
        """category / brand / product のいずれかを返す"""
        lname = name.lower()
        if lname.endswith("category_ytd") or lname.endswith("category_month"):
            return "category"
        if lname.endswith("brand_ytd") or lname.endswith("brand_month"):
            return "brand"
        if lname.endswith("product_ytd") or lname.endswith("product_month"):
            return "product"
        # デフォルトは brand とみなす（必要なら変更）
        return "brand"

    for sheet_name, df in dfs_dict.items():
        if df is None or df.empty:
            print(f"(skip) {sheet_name}: empty input")
            continue

        kind = _sheet_kind(sheet_name)

        # 入力前処理
        df2 = df.copy()

        # 不要列の削除（あれば）
        if "SourceFile" in df2.columns:
            df2 = df2.drop(columns=["SourceFile"])

        # 軽い整形
        df2.columns = [str(c).strip() for c in df2.columns]
        for c in df2.select_dtypes(include="object").columns:
            df2[c] = df2[c].astype(str).str.strip()

        # Period 正規化（"YYYY-MM"想定、"(YTD)"などを除去）
        if "Period" in df2.columns:
            df2["Period"] = df2["Period"].astype(str).str.replace(" (YTD)", "", regex=False).str.strip()
        else:
            df2["Period"] = ""

        # Segment / Subsegment は存在前提（無ければ作成）
        for col in ["Segment", "Subsegment"]:
            if col not in df2.columns:
                df2[col] = ""

        # sheet_name 列を補完
        if "sheet_name" not in df2.columns:
            df2["sheet_name"] = sheet_name
        else:
            df2["sheet_name"] = (
                df2["sheet_name"]
                .astype(str)
                .replace({"": sheet_name, "nan": sheet_name})
                .fillna(sheet_name)
            )

        # 数値変換（ヘッダー/装飾は NaN に）
        for c in [c for c in numeric_candidates if c in df2.columns]:
            df2[c] = pd.to_numeric(df2[c], errors="coerce")

        # “数値候補がすべてNaN”の行は削除（完全な装飾/ゴミ行の排除）
        num_cols_exist = [c for c in numeric_candidates if c in df2.columns]
        if num_cols_exist:
            df2 = df2[~df2[num_cols_exist].isna().all(axis=1)]

        # 既存シートの読み込み or 新規作成
        try:
            ws = sh.worksheet(sheet_name)
        except gspread.WorksheetNotFound:
            ws = sh.add_worksheet(title=sheet_name, rows=2000, cols=30)
            print(f"🆕 '{sheet_name}' シートを新規作成")

        try:
            existed = get_as_dataframe(ws).dropna(how="all")
            existed.columns = [str(c).strip() for c in existed.columns]
        except Exception:
            existed = pd.DataFrame()

        # 連結（縦）
        combined = pd.concat([existed, df2], ignore_index=True, sort=False)

        # ---- 重複キー定義（シート種別ごと）----
        # 共通キー
        key_cols = ["Period", "Segment", "Subsegment", "sheet_name"]
        # Brand/Product を種別で追加
        if kind == "brand":
            if "Brand" in combined.columns:
                key_cols = ["Period", "Brand", "Segment", "Subsegment", "sheet_name"]
        elif kind == "product":
            if "Product" in combined.columns:
                key_cols = ["Period", "Product", "Segment", "Subsegment", "sheet_name"]
        else:  # category
            # 既定の共通キーのまま（Brand/Product は存在しない前提）
            pass

        # 重複除去（同一キーの最後=新しい行を採用）
        combined = combined.drop_duplicates(subset=[c for c in key_cols if c in combined.columns], keep="last")

        # Period_dt 作成（ソート用）
        combined["Period_dt"] = pd.to_datetime(combined["Period"], format="%Y-%m", errors="coerce")

        # ---- 追加：Grand Total と All の優先キー ----
        seg_norm = combined["Segment"].astype(str).str.strip().str.lower()
        subseg_norm = combined["Subsegment"].astype(str).str.strip().str.lower()

        # Segment: "Grand Total" を先頭に
        combined["__seg_rank"] = (~seg_norm.eq("grand total")).astype(int)  # Grand Total=0, その他=1

        # Subsegment: "All" を先頭に
        combined["__subseg_rank"] = (~subseg_norm.eq("all")).astype(int)    # All=0, その他=1

        # Period→Segment→Subsegmentの順に並び替え
        combined = combined.sort_values(
            by=["Period_dt", "__seg_rank", "Segment", "__subseg_rank", "Subsegment"],
            ascending=[False, True, True, True, True],
            na_position="last"
        ).reset_index(drop=True)

        # 後片付け
        combined = combined.drop(columns=["__seg_rank", "__subseg_rank"], errors="ignore")

        # 出力列を desired_columns のみに揃える
        output_cols = [c for c in desired_columns if c in combined.columns]
        combined = combined[output_cols]

        # 型の統一：数値列は数値、他は文字列
        # （※ 出力対象列だけを対象にする）
        for c in [c for c in numeric_candidates if c in combined.columns]:
            combined[c] = pd.to_numeric(combined[c], errors="coerce")
        for c in combined.columns:
            if c not in numeric_candidates:
                combined[c] = combined[c].astype(str)

        # 書き込み（全クリア→上書き）
        ws.clear()
        set_with_dataframe(ws, combined)

        print(f"✅ {sheet_name}: written {len(combined)} rows (dedup & sorted)")

    print("🎉 すべてのシートで追記＋重複除去＋列順統一＋指定ソートを完了しました")

append_to_gsheet_with_dedup_v2(dfs, GSHEET_NAME)

📗 Opened: NPD（スキンケアテスト用）
✅ skincare_category_month: written 80 rows (dedup & sorted)
✅ skincare_category_ytd: written 80 rows (dedup & sorted)
✅ skincare_brand_month: written 1771 rows (dedup & sorted)
✅ skincare_brand_ytd: written 1789 rows (dedup & sorted)
✅ skincare_product_month: written 3151 rows (dedup & sorted)
✅ skincare_product_ytd: written 3190 rows (dedup & sorted)
✅ hair_category_month: written 40 rows (dedup & sorted)
✅ hair_category_ytd: written 40 rows (dedup & sorted)
✅ hair_brand_month: written 1134 rows (dedup & sorted)
✅ hair_brand_ytd: written 1147 rows (dedup & sorted)
✅ hair_product_month: written 2074 rows (dedup & sorted)
✅ hair_product_ytd: written 2104 rows (dedup & sorted)
✅ makeup_category_month: written 180 rows (dedup & sorted)
✅ makeup_category_ytd: written 180 rows (dedup & sorted)
✅ makeup_brand_month: written 4018 rows (dedup & sorted)
✅ makeup_brand_ytd: written 4066 rows (dedup & sorted)
✅ makeup_product_month: written 7447 rows (dedup & sorted)
✅ ma

In [ ]:
# ===== 検証セル：GSheetソース＋ラベル尊重＋サプレッション対応（フル） =====
import pandas as pd, numpy as np, re, os, glob

# ---------------- Config（ここだけ触れば挙動を変えられる） ----------------
CFG = {
    # Excelは SOURCE="excel" のときだけ有効
    "EXCEL_PATHS": "/content/brand_product_all.xlsx",

    # ▼ データソース
    "SOURCE": "gsheet",                       # "gsheet" または "excel"
    "GSHEET_NAME":  "NPD（スキンケアテスト用）",  # ★あなたのブック名に変更

    # ▼ GSheetのラベル（Segment/Subsegment）を基本的に信頼するか
    "RESPECT_GSHEET_LABELS": True,

    # 許容差
    "ATOL_DOLLARS": 0.01,
    "ATOL_UNITS":   0.1,
    "SHOW_ROWS":    10,

    # 列名ゆらぎ（小文字で比較）
    "COL_ALIASES": {
        "dollars": {"dollars","dollar sales","sales dollars"},
        "units":   {"units","unit sales","sales units"},
        "brand":   {"brand","brands"},
        "product": {"product","products"},
        "segment": {"segment"},
        "subsegment": {"subsegment","sub-segment"},
        "period": {"period"},
        "sheet_name": {"sheet_name"},
    },

    # 追加の微調整マップ（両側に適用・GSheet尊重モード時は使わない）
    "EXTRA_SUB_REPLACE": {
        r"\b&\b": "AND",
        r"\u3000": " ",         # 全角スペース → 半角
        r"[‐-‒–—−]": "-",       # ダッシュ類 → "-"
        r"\s*-\s*": "-",        # "-" 周りの空白除去
        r"\s*/\s*": "/",        # "/" 周りの空白除去
        r"\(|\)": "",           # 括弧除去（必要に応じて）
    },

    # 正規化マップ（両側に適用・GSheet尊重モード時は使わない）
    "COMMON_MAP": {
        r"\bACC\b": "ACCESSORIES",
        r"^ALL OTHER ": "OTHER ",
        r"\bSETS TOOLS AND ACCESSORIES\b": "SET TOOLS AND ACCESSORIES",
    },
    # category 側だけに適用（GSheet尊重モード時は使わない）
    "CAT_MAP": {
        r"SETTING SPRAY POWDER": "SETTING PRODUCT",
        r"BASE COATS TOP COATS": "BASE TOP COATS",
    },
    # category 側の (SEG,SUB) → SUB 読み替え（GSheet尊重モード時は使わない）
    "CAT_PAIR_OVERRIDE": {
        ("ALL OTHER", "ALL"): "OTHER COLOR",
    },
    # brand/product 側の特例読み替え（product のみ・GSheet尊重モード時は使わない）
    "BP_PRODUCT_PAIR_OVERRIDE": {
        ("ALL OTHER", "ALL"): "OTHER COLOR",
    },

    # category 側で比較対象から除外するペア（(SEG,SUB)）
    "CAT_DROP_RULES": {("*", "ALL"), ("SETS", "SET")},

    # 検証結果の「right_only（bp側欠落）」表示から除外するキー（(SEG,SUB)）
    "EXCLUDE_MISSING_RIGHT_ONLY": {("ALL OTHER", "OTHER COLOR")},

    # 任意：両側の比較から完全除外したい (SEG,SUB) キー（ワイルドカード "*" 可）
    "IGNORE_SUBS": set(),  # 例：{("*","OTHER COLOR"), ("SETS","SET")}

    # ▼▼ 追加：出力抑止ルール（bad/miss_bp/miss_cat それぞれに適用）▼▼
    # 形式は 2通り：
    #  (SEG, SUB)                 … 全ドメイン/全期間に適用（ワイルドカード "*" 可）
    #  (DOMAIN, SUFFIX, SEG, SUB) … 例: ("hair","month","TREATMENT","TONER/GLAZE")
    "SUPPRESS_BAD": set(),
    "SUPPRESS_MISS_BP": set(),   # left_only（= category に欠落）行を隠す
    "SUPPRESS_MISS_CAT": set(),  # right_only（= bp に欠落）行を隠す

    # チェック対象のドメインとサフィックス
    "DOMAINS":  ["makeup","skincare","hair"],
    "SUFFIXES": ["month","ytd"],
}

# ========= 既知の差分をサプレッション（CFGの直下に追記済み） =========

# 初期化（無ければ空集合を用意）
CFG.setdefault("SUPPRESS_BAD", set())
CFG.setdefault("SUPPRESS_MISS_BP", set())
CFG.setdefault("SUPPRESS_MISS_CAT", set())

# 3種（bad / left_only / right_only）すべてに同じルールを入れる関数
def _suppress_all(rule_set):
    CFG["SUPPRESS_BAD"]      |= rule_set
    CFG["SUPPRESS_MISS_BP"]  |= rule_set
    CFG["SUPPRESS_MISS_CAT"] |= rule_set

rules = set()

# --- skincare ドメイン ---
# SEG=SETS&KITS, SUB=SETS/KITS
rules |= {
    ("skincare", "*", "SETS & KITS", "SETS/KITS"),
}

# --- hair ドメイン ---
# 指定に基づき、typo保険（TRATMENT）も併記
rules |= {
    ("hair", "*", "TREATMENT", "ALL OTHER TREATMENT"),
    ("hair", "*", "TRATMENT",  "ALL OTHER TRATMENT"),
    ("hair", "*", "TREATMENT", "LEAVE-IN/SCALP TREATMENT"),
    ("hair", "*", "TRATMENT",  "LEAVE-IN/SCALP TREATMENT"),
    ("hair", "*", "TREATMENT", "MASK"),
    ("hair", "*", "TRATMENT",  "MASK"),
    ("hair", "*", "TREATMENT", "OIL"),
    ("hair", "*", "TREATMENT", "SERUM"),
    ("hair", "*", "SETS", "SETS"),
}

# 3種類すべて（bad / miss_bp / miss_cat）に反映
_suppress_all(rules)

# ---------------- Utils ----------------
def _expand_paths(paths):
    if isinstance(paths, str):
        if any(ch in paths for ch in "*?[]"):
            return sorted(glob.glob(paths))
        return [paths]
    return list(paths)

def _col(df: pd.DataFrame, logical: str):
    cand = CFG["COL_ALIASES"][logical]
    lower = {str(c).lower(): c for c in df.columns}
    for x in cand:
        if x in lower:
            return lower[x]
    return None

def _num(df, cols):
    """カンマ/空白除去 → 数値化（失敗はNaN）"""
    df = df.copy()
    for c in cols:
        if c and c in df.columns:
            df[c] = df[c].astype(str).str.replace(",", "").str.strip()
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def _lite_norm(s: pd.Series) -> pd.Series:
    """GSheetラベルを尊重：大文字化＋空白正規化のみ"""
    return (s.astype(str)
             .str.replace(r"\s+", " ", regex=True)
             .str.strip()
             .str.upper())

def _norm_txt(s: pd.Series, for_cat: bool=False) -> pd.Series:
    """比較キー生成のための正規化。GSheet尊重モードでは軽量化"""
    if CFG.get("RESPECT_GSHEET_LABELS", False) and CFG.get("SOURCE","").lower()=="gsheet":
        return _lite_norm(s)

    t = s.astype(str).str.strip()
    t = t.str.replace(r"\s+", " ", regex=True)
    t = t.str.replace("&", "AND")
    t = t.str.replace(r"[/_\-]", " ", regex=True)
    t = t.str.replace(r"[’'`]", "", regex=True).str.upper()
    for pat, repl in CFG["COMMON_MAP"].items():
        t = t.str.replace(pat, repl, regex=True)
    if for_cat:
        for pat, repl in CFG["CAT_MAP"].items():
            t = t.str.replace(pat, repl, regex=True)
    for pat, repl in CFG.get("EXTRA_SUB_REPLACE", {}).items():
        t = t.str.replace(pat, repl, regex=True)
    return t.str.replace(r"\s+", " ", regex=True).str.strip()

def _grand_total_mask(df, label_col):
    if label_col not in df.columns:
        return pd.Series(False, index=df.index)
    lab = _norm_txt(df[label_col])
    return lab.isin({"GRAND TOTAL","TOTAL","TOTAL ALL BRANDS","TOTAL - ALL BRANDS"})

def _summarize(title, bad=None, miss_bp=None, miss_cat=None, cols=None):
    if (bad is None or bad.empty) and (miss_bp is None or miss_bp.empty) and (miss_cat is None or miss_cat.empty):
        print(f"✅ {title}: すべて一致"); return
    print(f"❌ {title}: 不一致 / 欠落あり")
    if bad is not None and not bad.empty:
        print("  ├─ 不一致（上位表示）")
        display(bad[ [c for c in (cols or bad.columns) if c in bad.columns] ].head(CFG["SHOW_ROWS"]))
    if miss_bp is not None and not miss_bp.empty:
        print("  ├─ category に欠落（brand/product のみ存在）")
        display(miss_bp.head(CFG["SHOW_ROWS"]))
    if miss_cat is not None and not miss_cat.empty:
        print("  └─ brand/product に欠落（category のみ存在）")
        display(miss_cat.head(CFG["SHOW_ROWS"]))

def _load_all():
    """
    1) 既に globals()['dfs'] があればそれを使う
    2) SOURCE="gsheet" の場合は Googleスプレッドシートから全ワークシート読込
    3) SOURCE="excel" の場合は Excel から読込
    """
    g = globals()
    if "dfs" in g and isinstance(g["dfs"], dict) and g["dfs"]:
        return g["dfs"]

    source = CFG.get("SOURCE", "excel").lower()

    if source == "gsheet":
        # === Google Sheets から全シート読込 ===
        import gspread
        from gspread_dataframe import get_as_dataframe

        name = CFG.get("GSHEET_NAME")
        if not name:
            print("⚠️ CFG['GSHEET_NAME'] が未設定です"); return {}

        try:
            sh = gc.open(name)
            print(f"📗 Opened GSheet: {name}")
        except gspread.SpreadsheetNotFound:
            print(f"⚠️ GSheet が見つかりません: {name}")
            return {}

        out = {}
        for ws in sh.worksheets():
            try:
                df = get_as_dataframe(ws).dropna(how="all")
                # 列名整形
                df.columns = [str(c).strip() for c in df.columns]
                # Period 装飾除去（念のため）
                if "Period" in df.columns:
                    df["Period"] = (
                        df["Period"].astype(str)
                        .str.replace(" (YTD)", "", regex=False)
                        .str.strip()
                    )
                out[ws.title] = df
            except Exception as e:
                print(f"⚠️ 読み込み失敗: {ws.title} -> {e}")
        return out

    # === Excel から読込 ===
    paths = _expand_paths(CFG["EXCEL_PATHS"])
    if not paths:
        print("⚠️ EXCEL_PATHS が見つかりません"); return {}

    out = {}
    for p in paths:
        try:
            xls = pd.ExcelFile(p)
            for name in xls.sheet_names:
                df = pd.read_excel(p, sheet_name=name)
                if name in out and not out[name].empty:
                    out[name] = pd.concat([out[name], df], ignore_index=True)
                else:
                    out[name] = df
        except Exception as e:
            print(f"⚠️ 読み込みエラー: {p} -> {e}")
    return out

# ---------------- 共通フィルタ（任意の除外ルール） ----------------
def _should_drop_pair(seg, sub):
    """CFG['IGNORE_SUBS'] に合致する (SEG,SUB) は除外"""
    rules = CFG.get("IGNORE_SUBS", set()) or set()
    if not rules: return False
    for s,u in rules:
        if (s=="*" or s==seg) and (u=="*" or u==sub):
            return True
    return False

# ▼▼ サプレッション・ヘルパー ▼▼
def _pair_match(seg, sub, rules, domain=None, suffix=None):
    """(SEG,SUB) が rules に一致するか。rules要素は (seg,sub) or (domain,suffix,seg,sub)。'*' ワイルドカード可。"""
    if not rules:
        return False
    seg = str(seg); sub = str(sub)
    for r in rules:
        if len(r) == 2:
            rs, ru = r
            if (rs in ("*", seg)) and (ru in ("*", sub)):
                return True
        elif len(r) == 4:
            rd, rf, rs, ru = r
            if (rd in ("*", domain)) and (rf in ("*", suffix)) and (rs in ("*", seg)) and (ru in ("*", sub)):
                return True
    return False

def _suppress_rows(df, rules, domain, suffix):
    """DataFrame（SEG/SUB列を含む）から、rules に一致する行を除去"""
    if df is None or df.empty or not rules:
        return df
    mask = df.apply(lambda r: _pair_match(r.get("SEG"), r.get("SUB"), rules, domain, suffix), axis=1)
    return df.loc[~mask].copy()

# ---------------- 前処理（共通化） ----------------
def _prepare_bp(df: pd.DataFrame, who: str) -> pd.DataFrame:
    """brand/product から Grand Total を (Period, SEG, SUB) で集計（TOTAL接頭辞なし設計）"""
    if df is None or df.empty: return pd.DataFrame()
    df = df.copy()

    c_seg = _col(df, "segment")
    c_sub = _col(df, "subsegment")
    c_per = _col(df, "period")
    c_dol = _col(df, "dollars")
    c_uni = _col(df, "units")
    if not all([c_seg, c_sub, c_per, c_dol, c_uni]):
        return pd.DataFrame()

    # Dollars空欄除外（0は残す）
    df = df[df[c_dol].notna() & (df[c_dol].astype(str).str.strip() != "")]
    df = _num(df, [c_dol, c_uni])

    df["SEG"] = _norm_txt(df[c_seg])
    df["SUB"] = _norm_txt(df[c_sub])
    df["PER"] = df[c_per].astype(str)

    # Segment='ALL' は比較対象外
    df = df[~df["SEG"].eq("ALL")]

    # Grand Total のみ
    label = _col(df, "brand") if who=="brand" else _col(df, "product")
    if not label: return pd.DataFrame()
    df = df[_grand_total_mask(df, label)]
    if df.empty: return df

    # product 側の特例読み替え（GSheet尊重モードならスキップ）
    if (who == "product") and not (CFG.get("RESPECT_GSHEET_LABELS", False) and CFG.get("SOURCE","").lower()=="gsheet"):
        pairs = list(zip(df["SEG"], df["SUB"]))
        df["SUB"] = [CFG["BP_PRODUCT_PAIR_OVERRIDE"].get(p, s) for p, s in zip(pairs, df["SUB"])]

    # “ALL/空/NAN” は比較対象外
    df = df[df["SUB"].notna() & (~df["SUB"].isin({"ALL","","NAN"}))]

    # 任意除外（IGNORE_SUBS）
    df = df[~df.apply(lambda r: _should_drop_pair(r["SEG"], r["SUB"]), axis=1)]

    out = (df.groupby(["PER","SEG","SUB"], dropna=False)[[c_dol, c_uni]]
             .sum(numeric_only=True).reset_index())
    return out.rename(columns={"PER":"Period","SEG":"SEG","SUB":"SUB",
                               c_dol:"Dollars", c_uni:"Units"})

def _prepare_cat(df: pd.DataFrame) -> pd.DataFrame:
    """*_category_* を (Period, SEG, SUB) で集計（読み替え＋除外）"""
    if df is None or df.empty: return pd.DataFrame()
    df = df.copy()

    c_seg = _col(df, "segment")
    c_sub = _col(df, "subsegment")
    c_per = _col(df, "period")
    c_dol = _col(df, "dollars")
    c_uni = _col(df, "units")
    if not all([c_seg, c_sub, c_per, c_dol, c_uni]):
        return pd.DataFrame()

    # Dollars空欄除外（0は残す）
    df = df[df[c_dol].notna() & (df[c_dol].astype(str).str.strip() != "")]
    df = _num(df, [c_dol, c_uni])

    df["SEG"] = _norm_txt(df[c_seg], for_cat=True)
    df["SUB"] = _norm_txt(df[c_sub], for_cat=True)
    df["PER"] = df[c_per].astype(str)

    # ペア読み替え（GSheet尊重モードならスキップ）
    if not (CFG.get("RESPECT_GSHEET_LABELS", False) and CFG.get("SOURCE","").lower()=="gsheet"):
        pairs = list(zip(df["SEG"], df["SUB"]))
        df["SUB"] = [CFG["CAT_PAIR_OVERRIDE"].get(p, s) for p, s in zip(pairs, df["SUB"])]

    # category側の除外ルール
    def _drop(seg, sub):
        for s,u in CFG["CAT_DROP_RULES"]:
            if (s=="*" or s==seg) and (u=="*" or u==sub):
                return True
        return False
    df = df[~df.apply(lambda r: _drop(r["SEG"], r["SUB"]), axis=1)]

    # 任意除外（IGNORE_SUBS）
    df = df[~df.apply(lambda r: _should_drop_pair(r["SEG"], r["SUB"]), axis=1)]

    out = (df.groupby(["PER","SEG","SUB"], dropna=False)[[c_dol, c_uni]]
             .sum(numeric_only=True).reset_index())
    return out.rename(columns={"PER":"Period","SEG":"SEG","SUB":"SUB",
                               c_dol:"Dollars", c_uni:"Units"})

# ---------------- チェック ----------------
def check_brand_vs_product_totals(dfs_dict, domain, suffix):
    bm_key, pm_key = f"{domain}_brand_{suffix}", f"{domain}_product_{suffix}"
    bm, pm = dfs_dict.get(bm_key, pd.DataFrame()), dfs_dict.get(pm_key, pd.DataFrame())
    if bm.empty or pm.empty:
        print(f"⚠️ {bm_key} / {pm_key} が空。スキップ"); return
    bm, pm = _num(bm, [_col(bm,"dollars"), _col(bm,"units")]), _num(pm, [_col(pm,"dollars"), _col(pm,"units")])

    def _gt(df, label_logical):
        label = _col(df, label_logical)
        if not label: return pd.DataFrame(columns=["Dollars","Units"])
        m = _grand_total_mask(df, label)
        c_per, c_dol, c_uni = _col(df,"period"), _col(df,"dollars"), _col(df,"units")
        if not all([c_per, c_dol, c_uni]): return pd.DataFrame()
        return (df.loc[m].groupby(c_per)[[c_dol, c_uni]].sum(numeric_only=True)
                  .rename(columns={c_dol:"Dollars", c_uni:"Units"}))

    both = _gt(bm,"brand").join(_gt(pm,"product"), how="outer", lsuffix="_brand", rsuffix="_product").fillna(0.0)

    ok = (both["Dollars_brand"].sub(both["Dollars_product"]).abs() <= CFG["ATOL_DOLLARS"]) & \
         (both["Units_brand"].sub(both["Units_product"]).abs()   <= CFG["ATOL_UNITS"])
    if ok.all():
        print(f"✅ {bm_key} vs {pm_key} Grand Total は Periodごとに一致")
    else:
        bad = both.loc[~ok].assign(
            Dollars_diff=lambda d: d["Dollars_brand"]-d["Dollars_product"],
            Units_diff=lambda d: d["Units_brand"]-d["Units_product"]
        )
        _summarize(f"{bm_key} vs {pm_key} Grand Total", bad=bad)

EXCLUDE_MISSING_RIGHT_ONLY = set(CFG["EXCLUDE_MISSING_RIGHT_ONLY"])

def _filter_missing_right_only(df_missing: pd.DataFrame, who: str):
    if df_missing.empty or who != "product": return df_missing
    keys = list(zip(df_missing.get("SEG", pd.Series([])), df_missing.get("SUB", pd.Series([]))))
    mask = pd.Series([(k in EXCLUDE_MISSING_RIGHT_ONLY) for k in keys], index=df_missing.index)
    return df_missing[~mask]

def check_bp_vs_category(dfs_dict, domain, suffix):
    cat_key = f"{domain}_category_{suffix}"
    cat = _prepare_cat(dfs_dict.get(cat_key, pd.DataFrame()))
    if cat.empty:
        print(f"⚠️ {cat_key} が空。スキップ"); return

    for who in ["brand","product"]:
        bp_key = f"{domain}_{who}_{suffix}"
        bp_raw = dfs_dict.get(bp_key, pd.DataFrame())
        bp     = _prepare_bp(bp_raw, who)
        ttl    = f"{bp_key} ⇄ {cat_key}"
        if bp.empty:
            print(f"⚠️ {ttl}: {bp_key} 側が空。スキップ"); continue

        merged = bp.merge(cat, on=["Period","SEG","SUB"], how="outer",
                          suffixes=("_bp","_cat"), indicator=True)

        miss_cat = merged[merged["_merge"].eq("left_only")].copy()   # category に欠落
        miss_bp  = merged[merged["_merge"].eq("right_only")].copy()  # bp に欠落
        miss_bp  = _filter_missing_right_only(miss_bp, who)

        # ★ サプレッション適用（欠落系）
        miss_cat = _suppress_rows(miss_cat, CFG.get("SUPPRESS_MISS_BP", set()), domain, suffix)
        miss_bp  = _suppress_rows(miss_bp,  CFG.get("SUPPRESS_MISS_CAT", set()), domain, suffix)

        both = merged[merged["_merge"].eq("both")].copy()
        both["Dollars_abs_diff"] = (both["Dollars_bp"] - both["Dollars_cat"]).abs()
        both["Units_diff"]       = (both["Units_bp"]   - both["Units_cat"]).abs()
        bad = both[(both["Dollars_abs_diff"] > CFG["ATOL_DOLLARS"]) |
                   (both["Units_diff"]       > CFG["ATOL_UNITS"])]

        # ★ サプレッション適用（不一致）
        bad = _suppress_rows(bad, CFG.get("SUPPRESS_BAD", set()), domain, suffix)

        if bad.empty and miss_cat.empty and miss_bp.empty:
            print(f"✅ {ttl}: すべて一致")
        else:
            cols = ["Period","SEG","SUB","Dollars_bp","Dollars_cat","Dollars_abs_diff","Units_bp","Units_cat","Units_diff"]
            _summarize(ttl,
                       bad=bad[cols] if not bad.empty else bad,
                       miss_bp=miss_cat[["Period","SEG","SUB","Dollars_bp","Units_bp"]] if not miss_cat.empty else miss_cat,
                       miss_cat=miss_bp[["Period","SEG","SUB","Dollars_cat","Units_cat"]] if not miss_bp.empty else miss_bp)

# ---------------- 実行 ----------------
# ※ もし前のセッションで dfs が残っているとExcel等が混ざるので必ず消す
try:
    del dfs
except NameError:
    pass

dfs_all = _load_all()

print("🔎 SOURCE:", CFG.get("SOURCE"), "GSHEET_NAME:", CFG.get("GSHEET_NAME"))
print("🧾 読み込んだシート一覧（先頭10件）:", list(dfs_all.keys())[:10])

print("\n🤝 brand vs product Grand Total 一致チェック（全ドメイン＆月/YTD）")
for dom in CFG["DOMAINS"]:
    for suf in CFG["SUFFIXES"]:
        check_brand_vs_product_totals(dfs_all, dom, suf)

print("\n🔗 Grand Total（brand/product）⇄ category 突合（Segment/Subsegment, Period一致）")
for dom in CFG["DOMAINS"]:
    for suf in CFG["SUFFIXES"]:
        check_bp_vs_category(dfs_all, dom, suf)

print("\n✅ 検証完了（❌ が出た箇所だけ重点確認してください）")

📗 Opened GSheet: NPD（スキンケアテスト用）
🔎 SOURCE: gsheet GSHEET_NAME: NPD（スキンケアテスト用）
🧾 読み込んだシート一覧（先頭10件）: ['skincare_category_month', 'skincare_category_ytd', 'skincare_brand_month', 'skincare_brand_ytd', 'skincare_product_month', 'skincare_product_ytd', 'hair_category_month', 'hair_category_ytd', 'hair_brand_month', 'hair_brand_ytd']

🤝 brand vs product Grand Total 一致チェック（全ドメイン＆月/YTD）
✅ makeup_brand_month vs makeup_product_month Grand Total は Periodごとに一致
✅ makeup_brand_ytd vs makeup_product_ytd Grand Total は Periodごとに一致
✅ skincare_brand_month vs skincare_product_month Grand Total は Periodごとに一致
✅ skincare_brand_ytd vs skincare_product_ytd Grand Total は Periodごとに一致
✅ hair_brand_month vs hair_product_month Grand Total は Periodごとに一致
✅ hair_brand_ytd vs hair_product_ytd Grand Total は Periodごとに一致

🔗 Grand Total（brand/product）⇄ category 突合（Segment/Subsegment, Period一致）
✅ makeup_brand_month ⇄ makeup_category_month: すべて一致
✅ makeup_product_month ⇄ makeup_category_month: すべて一致
✅ makeup_brand_ytd ⇄ makeu